In [ ]:
import tensorflow as tf 
import pathlib
import numpy as np
from matplotlib import pyplot
from matplotlib import image
import glob

In [ ]:
def download_oasis ():
    
    #download oasis brain MRI data
    dataset_url = "https://cloudstor.aarnet.edu.au/plus/s/n5aZ4XX1WBKp6HZ/download"
    data_dir = tf.keras.utils.get_file(origin=dataset_url,fname='oa-sis' ,untar=True)
    data_dir = pathlib.Path(data_dir)
    
    # unzip data to current directory 
    print (data_dir)
    ! unzip /root/.keras/datasets/oa-sis.tar.gz

In [ ]:
def load_training (path):
    # load training images (non segmented) in the path and store in numpy array
    image_list = []
    for filename in glob.glob(path+'/*.png'): 
        im=image.imread (filename)
        image_list.append(im)

    print('train_X shape:',np.array(image_list).shape)
    train_set = np.array(image_list, dtype=np.float32)
    return train_set


In [ ]:
def process_training (data_set):
    # the method normalizes training images and adds 4th dimention 

    train_set = data_set
    train_set = (train_set - np.mean(train_set))/ np.std(train_set)
    train_set= (train_set- np.amin(train_set))/ np.amax(train_set- np.amin(train_set))
    train_set = train_set [:,:,:,np.newaxis]
    
    return train_set

In [ ]:
def load_labels (path):
    # loads labels images and map pixel values to class indices and convert image data type to unit8 

    n_classes = 4
    image_list =[]
    for filename in glob.glob(path+'/*.png'): 
        im=image.imread (filename)
        one_hot = np.zeros((im.shape[0], im.shape[1]))
        for i, unique_value in enumerate(np.unique(im)):
          one_hot[:, :][im == unique_value] = i
        image_list.append(one_hot)

    print('train_y shape:',np.array(image_list).shape)
    labels = np.array(image_list, dtype=np.uint8)
    
    pyplot.imshow(labels[2])
    pyplot.show()

    return labels

In [ ]:
def process_labels(seg_data):
    # one hot encode label data and convert to numpy array
    onehot_Y = []
    for n in range(seg_data.shape[0]): 
      im = seg_data[n]
      n_classes = 4
      one_hot = np.zeros((im.shape[0], im.shape[1], n_classes),dtype=np.uint8)
      for i, unique_value in enumerate(np.unique(im)):
          one_hot[:, :, i][im == unique_value] = 1
      onehot_Y.append(one_hot)
    
    onehot_Y =np.array(onehot_Y)
    print (onehot_Y.dtype)
    #print (np.unique(onehot_validate_Y))
    print (onehot_Y.shape)

    return onehot_Y


In [5]:
# U-NET final model w on2Dtranspose and batch normalization after activation
# questions do I need to use shuffle =true in fit module ?
import tensorflow as tf

def unet_model ():
    filter_size=16 
    input_layer = tf.keras.Input((256,256,1))
    
    pre_conv = tf.keras.layers.Conv2D(filter_size * 1, (3, 3), padding="same")(input_layer)
    pre_conv = tf.keras.layers.LeakyReLU(alpha=.01)(pre_conv)


# context module 1 pre-activation residual block
    conv1 = tf.keras.layers.BatchNormalization()(pre_conv)
    conv1 = tf.keras.layers.LeakyReLU(alpha=.01)(conv1)
    conv1 = tf.keras.layers.Conv2D(filter_size * 1, (3, 3), padding="same" )(conv1) 
    conv1 = tf.keras.layers.Dropout(.3) (conv1)
    conv1 = tf.keras.layers.BatchNormalization()(conv1)
    conv1 = tf.keras.layers.LeakyReLU(alpha=.01)(conv1)
    conv1 = tf.keras.layers.Conv2D(filter_size * 1, (3, 3), padding="same")(conv1)    
    conv1 = tf.keras.layers.Add()([pre_conv,conv1])
    
# downsample and double number of feature maps   
    pool1 = tf.keras.layers.Conv2D(filter_size * 2, (3,3), (2,2) , padding='same')(conv1)
    pool1 = tf.keras.layers.LeakyReLU(alpha=.01)(pool1)
    
# context module 2
    conv2 = tf.keras.layers.BatchNormalization()(pool1)
    conv2 = tf.keras.layers.LeakyReLU(alpha=.01)(conv2)
    conv2 = tf.keras.layers.Conv2D(filter_size * 2, (3, 3), padding="same")(conv2)
    conv2 = tf.keras.layers.Dropout(.3) (conv2)  
    conv2 = tf.keras.layers.BatchNormalization()(conv2)
    conv2 = tf.keras.layers.LeakyReLU(alpha=.01)(conv2)
    conv2 = tf.keras.layers.Conv2D(filter_size * 2, (3, 3), padding="same")(conv2)
    conv2 = tf.keras.layers.Add()([pool1,conv2])

# downsample and double number of feature maps
    pool2 = tf.keras.layers.Conv2D(filter_size*4, (3,3),(2,2), padding='same')(conv2)
    pool2 = tf.keras.layers.LeakyReLU(alpha=.01)(pool2)

# context module 3
    conv3 = tf.keras.layers.BatchNormalization()(pool2)
    conv3 = tf.keras.layers.LeakyReLU(alpha=.01)(conv3)
    conv3 = tf.keras.layers.Conv2D(filter_size * 4, (3, 3), padding="same")(conv3)
    conv3 = tf.keras.layers.Dropout(.3) (conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)
    conv3 = tf.keras.layers.LeakyReLU(alpha=.01)(conv3)
    conv3 = tf.keras.layers.Conv2D(filter_size * 4, (3, 3), padding="same")(conv3)
    conv3 = tf.keras.layers.Add()([pool2,conv3])

# downsample and double number of feature maps
    pool3 = tf.keras.layers.Conv2D(filter_size*8, (3,3),(2,2),padding='same')(conv3)
    pool3 = tf.keras.layers.LeakyReLU(alpha=.01)(pool3)

# context module 4
    conv4 = tf.keras.layers.BatchNormalization()(pool3)
    conv4 = tf.keras.layers.LeakyReLU(alpha=.01)(conv4)
    conv4 = tf.keras.layers.Conv2D(filter_size * 8, (3, 3), padding="same")(conv4)
    conv4 = tf.keras.layers.Dropout(.3) (conv4)
    conv4 = tf.keras.layers.BatchNormalization()(conv4)
    conv4 = tf.keras.layers.LeakyReLU(alpha=.01)(conv4)
    conv4 = tf.keras.layers.Conv2D(filter_size * 8, (3, 3), padding="same")(conv4)
    conv4 = tf.keras.layers.Add()([pool3,conv4])
    print ("conv4",conv4.shape)

# downsample and double number of feature maps
    pool4 = tf.keras.layers.Conv2D(filter_size*16, (3,3),(2,2),padding='same')(conv4)
    pool4 = tf.keras.layers.LeakyReLU(alpha=.01)(pool4) 

# context module 5
    # Middle
    convm = tf.keras.layers.BatchNormalization()(pool4)
    convm = tf.keras.layers.LeakyReLU(alpha=.01)(convm)
    convm = tf.keras.layers.Conv2D(filter_size * 16, (3, 3), padding="same")(convm)
    convm = tf.keras.layers.Dropout(.3) (convm)
    convm = tf.keras.layers.BatchNormalization()(convm)
    convm = tf.keras.layers.LeakyReLU(alpha=.01)(convm)
    convm = tf.keras.layers.Conv2D(filter_size * 16, (3, 3), padding="same")(convm)
    convm = tf.keras.layers.Add()([pool4,convm])


#upsampling module 1
    deconv4 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='bilinear')(convm)
    deconv4 = tf.keras.layers.Conv2D (filter_size *8, (3, 3) , padding="same")(deconv4)
    deconv4 = tf.keras.layers.LeakyReLU(alpha=.01)(deconv4) 
    print ("upsample 1",deconv4.shape)

#concatatinate layers 
    uconv4 = tf.keras.layers.concatenate([deconv4, conv4], axis=3)


#localization module 1
    uconv4 = tf.keras.layers.Conv2D(filter_size * 16, (3, 3) , padding="same")(uconv4)
    uconv4 = tf.keras.layers.BatchNormalization()(uconv4)
    uconv4 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv4)
    uconv4 = tf.keras.layers.Conv2D(filter_size * 8, (1, 1), padding="same")(uconv4)
    uconv4 = tf.keras.layers.BatchNormalization()(uconv4)
    uconv4 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv4)

#upsampling module 2
    deconv3 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='bilinear')(uconv4)
    deconv3 = tf.keras.layers.Conv2D (filter_size *4, (3, 3) , padding="same")(deconv3)
    deconv3 = tf.keras.layers.LeakyReLU(alpha=.01)(deconv3) 

  

# concatatinate layers  
    uconv3 = tf.keras.layers.concatenate([deconv3, conv3], axis=3)


# localization module 2
    uconv3 = tf.keras.layers.Conv2D(filter_size * 8, (3, 3), padding="same")(uconv3)
    uconv3 = tf.keras.layers.BatchNormalization()(uconv3)
    uconv3 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv3)
    uconv3 = tf.keras.layers.Conv2D(filter_size * 4, (1, 1), padding="same")(uconv3)
    uconv3 = tf.keras.layers.BatchNormalization()(uconv3)
    uconv3 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv3)

# segmentation layer 1
    seg3 = tf.keras.layers.Conv2D(4, (3,3),  activation="softmax", padding='same' )(uconv3)
# upscale segmented layer 1
    seg3 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='bilinear')(seg3)


# Upsample module 3
    deconv2 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='bilinear')(uconv3)
    deconv2 = tf.keras.layers.Conv2D (filter_size *2, (3, 3) , padding="same")(deconv2)
    deconv2 = tf.keras.layers.LeakyReLU(alpha=.01)(deconv2)


# concatination layer 
    uconv2 = tf.keras.layers.concatenate([deconv2, conv2], axis=3)


# localization module 3
    uconv2 = tf.keras.layers.Conv2D(filter_size * 4, (3, 3), padding="same")(uconv2)
    uconv2 = tf.keras.layers.BatchNormalization()(uconv2)
    uconv2 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv2)
    uconv2 = tf.keras.layers.Conv2D(filter_size * 2, (1, 1), padding="same")(uconv2)
    uconv2 = tf.keras.layers.BatchNormalization()(uconv2)
    uconv2 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv2)

# segmentation layer 2
    seg2 = tf.keras.layers.Conv2D(4, (3,3),  activation="softmax", padding='same')(uconv2)

# add segmentation layer 1 and 2
    seg_32 = tf.keras.layers.Add()([seg3,seg2])
# upscale sum segmentation layer 1 and 2
    seg_32 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='bilinear')(seg_32)


# Upsample module 4
    deconv1 = tf.keras.layers.UpSampling2D(size=(2,2) , interpolation='bilinear')(uconv2)
    deconv1 = tf.keras.layers.Conv2D (filter_size *1, (3, 3) , padding="same")(deconv1)
    deconv1 = tf.keras.layers.LeakyReLU(alpha=.01)(deconv1)


# concatination layer
    uconv1 = tf.keras.layers.concatenate([deconv1, conv1], axis=3 )

#final convolution layer
    uconv1 = tf.keras.layers.Conv2D(filter_size * 2, (3, 3), padding="same")(uconv1)
    uconv1 = tf.keras.layers.BatchNormalization()(uconv1)
    uconv1 = tf.keras.layers.LeakyReLU(alpha=.01)(uconv1)
    
# final segmentation layer   
    seg1 = tf.keras.layers.Conv2D(4, (3,3),  activation="softmax", padding='same' )(uconv1)

# sum all segmentation layers 
    seg_sum = tf.keras.layers.Add()([seg1,seg_32])


    output_layer = tf.keras.layers.Conv2D(4, (3,3), padding='same' ,activation="softmax")(seg_sum)
    model = tf.keras.Model( input_layer , outputs=output_layer)

    
    return model




In [6]:
 
def dice_coefficient (y_true, y_pred):
  from keras import backend as k
  
  c1_true = k.flatten(y_true[:,:,:,3])
  c1_pred = k.flatten(y_pred[:,:,:,3]) 
  c2_true = k.flatten(y_true[:,:,:,2])
  c2_pred = k.flatten(y_pred[:,:,:,2]) 
  c3_true = k.flatten(y_true[:,:,:,1])
  c3_pred = k.flatten(y_pred[:,:,:,1]) 
  c4_true = k.flatten(y_true[:,:,:,0])
  c4_pred = k.flatten(y_pred[:,:,:,0]) 

  
  intersection1 = k.sum(c1_true*c1_pred)
  coeff1 = (2.0*intersection1)/(k.sum(c1_true) + k.sum(c1_pred) )

  intersection2 = k.sum(c2_true*c2_pred)
  coeff2 = (2.0*intersection2)/(k.sum(c2_true) + k.sum(c2_pred) )

  
  intersection3 = k.sum(c3_true*c3_pred)
  coeff3 = (2.0*intersection3)/(k.sum(c3_true) + k.sum(c3_pred) )

  intersection4 = k.sum(c4_true*c4_pred)
  coeff4 = (2.0*intersection4)/(k.sum(c4_true) + k.sum(c4_pred) )
  
  coeff= (coeff1+coeff2+coeff3+coeff4)/4.0
  return coeff
  
